## Imports

deep net RAE attack

deep net RAE iso-contours

denoising VAE preprocessor + MLP vs LCA + MLP

the single unit marzi attack

----------------------------
*network marzi attack*:
* Orthogonal to Marzi Untargeted should be the best iso-response direction
  * measure the curvature for all neurons in this direction
  * give avg for the marzi untargeted orthogonal direction vs random directions

*single layer marzi attack*:

v: user-provided (possibly unit) vector

a: activation vector of targeted layer

$max |v^Ta - v^T \tilde{a}|$

single unit: $v = e_i$

full layer: $v = \frac{1}{\sqrt(n)} \mathbb{1}$ (ones vector)


now we can do anything inbetween, e.g. attack only two neurons with high inner product
inspired from deep dreem lucid framework

In [ ]:
import os
os.chdir("../")
%env CUDA_VISIBLE_DEVICES=1
%matplotlib inline

In [ ]:
import pickle
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import tensorflow as tf
from data.dataset import Dataset
import data.data_selector as ds
import utils.data_processing as dp
import utils.plot_functions as pf
import analysis.analysis_picker as ap
#from modules.recon_adversarial_module import ReconAdversarialModule

In [ ]:
class ae_params(object):
  def __init__(self):
    self.model_type = "ae"
    self.model_name = "ae_768_mnist"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

class lca_params(object):
  def __init__(self):
    self.model_type = "lca"
    self.model_name = "lca_768_mnist"
    self.version = "0.0"
    self.save_info = "analysis_test_carlini_targeted"
    self.overwrite_analysis_log = False

params_list = [ae_params(), lca_params()]
for params in params_list:
  params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+params.model_name)

In [ ]:
analyzer_list = [ap.get_analyzer(params.model_type) for params in params_list]
for analyzer, params in zip(analyzer_list, params_list):
  analyzer.setup(params)
  analyzer.setup_model(analyzer.model_params)
  analyzer.load_analysis(save_info=params.save_info)
  analyzer.model_name = params.model_name

In [ ]:
# load data
dataset_type = "train"
data = ds.get_data(analyzer_list[0].model_params)
data = analyzer_list[0].model.preprocess_dataset(data, analyzer.model_params)
data = analyzer_list[0].model.reshape_dataset(data, analyzer.model_params)
data_min = data[dataset_type].images.min()
data_max = data[dataset_type].images.max()
dataset_size = data[dataset_type].images.shape[0]
print("NUM DATA", dataset_size, 'DATA MIN', data_min, 'DATA MAX', data_max)

In [ ]:
input_id = 5
adv_id = 1

all_adv_inputs = np.stack([data[dataset_type].images[img_id,...]
  for img_id in range(dataset_size)
  if dp.one_hot_to_dense(data[dataset_type].labels[img_id][None,...]) == input_id])
              
all_adv_targets = np.stack([data[dataset_type].images[img_id,...]
  for img_id in range(dataset_size)
  if dp.one_hot_to_dense(data[dataset_type].labels[img_id][None,...]) == adv_id])

if all_adv_inputs.shape[0] > all_adv_targets.shape[0]:
  all_adv_inputs = all_adv_inputs[:all_adv_targets.shape[0],...]
else:
  all_adv_targets = all_adv_targets[:all_adv_inputs.shape[0],...]


In [ ]:
img_id = 0
adv_inputs = all_adv_inputs[img_id,...][None,...]
adv_targets = all_adv_targets[img_id,...][None,...]

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(8,4))
ax = pf.clear_axis(axes[0])
ax.imshow(adv_inputs.reshape(28,28), cmap="Greys_r")
ax.set_title("Input image")
ax = pf.clear_axis(axes[1])
ax.imshow(adv_targets.reshape(28,28), cmap="Greys_r")
ax.set_title("Adv target image")
plt.show()

In [ ]:
#analyzer_list[0].analysis_params.adversarial_num_steps = int(1e2)
#analyzer_list[0].analysis_params.adversarial_attack_method = "kurakin_targeted"
#analyzer_list[0].analysis_params.adversarial_step_size = 1e-3

analyzer_list[0].analysis_params.adversarial_max_change = 1

#analyzer_list[0].analysis_params.adversarial_clip = False
#analyzer_list[0].analysis_params.adversarial_clip_range = [0.0, 1.0]
#analyzer_list[0].analysis_params.carlini_recon_mult = 2.6
#analyzer_list[0].analysis_params.carlini_change_variable = True
#analyzer_list[0].analysis_params.carlini_optimizer = "sgd"
#analyzer_list[0].analysis_params.adversarial_save_int = 1
  
analyzer_list[0].model.reset_graph()
analyzer_list[0].setup_model(analyzer_list[0].model_params) 

In [ ]:
feed_dict = analyzer_list[0].model.get_feed_dict(adv_inputs, is_test=True)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
with tf.Session(config=config, graph=analyzer_list[0].model.graph) as sess:
  sess.run(analyzer_list[0].model.init_op, feed_dict)
  analyzer_list[0].model.load_full_model(sess, analyzer_list[0].analysis_params.cp_loc)
  tensors = [analyzer_list[0].model.reconstruction]
  eval_list = sess.run(tensors, feed_dict)

In [ ]:
fig, ax = plt.subplots(1, figsize=(6,6))
ax = pf.clear_axis(ax)
ax.imshow(eval_list[0].reshape(28,28), cmap="Greys_r", vmin=0.0, vmax=1.0)
ax.set_title("recon")
plt.show()

In [ ]:
orig_recons = analyzer_list[0].evaluate_model(adv_inputs, ["output/reconstruction:0"])["output/reconstruction:0"]

In [ ]:
fig, ax = plt.subplots(1, figsize=(6,6))
ax = pf.clear_axis(ax)
ax.imshow(orig_recons.reshape(28,28), cmap="Greys_r", vmin=0.0, vmax=1.0)
ax.set_title("recon")
plt.show()

In [ ]:
#self.adv_loss =  input_pert_loss + self.recon_mult * adv_recon_loss

#AE
params_list[0].adversarial_num_steps = int(1e2)
params_list[0].adversarial_attack_method = "kurakin_targeted"#"marzi_untargeted"
params_list[0].adversarial_step_size = 1e-3
params_list[0].adversarial_max_change = None#0.3
params_list[0].adversarial_clip = False
params_list[0].adversarial_clip_range = [0.0, 1.0]
params_list[0].carlini_recon_mult = 2.6
params_list[0].carlini_change_variable = True
params_list[0].carlini_optimizer = "sgd"
params_list[0].adversarial_save_int = 1

#LCA
params_list[1].adversarial_num_steps = params_list[0].adversarial_num_steps#10
params_list[1].adversarial_attack_method = params_list[0].adversarial_attack_method#"carlini_targeted"
params_list[1].adversarial_step_size = params_list[0].adversarial_step_size#1e-5
params_list[1].adversarial_max_change = params_list[0].adversarial_max_change#0.2
params_list[1].adversarial_clip = params_list[0].adversarial_clip#False
params_list[1].adversarial_clip_range = params_list[0].adversarial_clip_range#[0.0, 1.0]
params_list[1].carlini_recon_mult = params_list[0].carlini_recon_mult#10
params_list[1].carlini_change_variable = params_list[0].carlini_change_variable#True
params_list[1].carlini_optimizer = params_list[0].carlini_optimizer#"adam"
params_list[1].adversarial_save_int = params_list[0].adversarial_save_int#1

In [ ]:
for analyzer, params in zip(analyzer_list, params_list):
  analyzer.analysis_params.adversarial_num_steps = params.adversarial_num_steps
  analyzer.analysis_params.adversarial_attack_method = params.adversarial_attack_method
  analyzer.analysis_params.adversarial_step_size = params.adversarial_step_size
  analyzer.analysis_params.adversarial_max_change = params.adversarial_max_change
  analyzer.analysis_params.adversarial_clip = params.adversarial_clip
  analyzer.analysis_params.adversarial_clip_range = params.adversarial_clip_range
  analyzer.analysis_params.carlini_recon_mult = params.carlini_recon_mult
  analyzer.analysis_params.carlini_change_variable = params.carlini_change_variable
  analyzer.analysis_params.carlini_optimizer = params.carlini_optimizer
  analyzer.analysis_params.adversarial_save_int = params.adversarial_save_int
  analyzer.model.reset_graph()
  analyzer.setup_model(analyzer.model_params) 

In [ ]:
"""
all_adversarial_images is a nested list [num_recon_mults][num_steps] and then an ndarray of shape (1, 784)
all_recons is a nested list [num_recon_mults][num_steps] and then an ndarray of shape (1, 784)
For Marzi attack, we need to compute original recons for every image
"""
for analyzer in analyzer_list:
  orig_recons = analyzer.evaluate_model(adv_inputs, ["output/reconstruction:0"])["output/reconstruction:0"]
  #analyzer.steps, analyzer.all_adversarial_images, analyzer.all_recons, analyzer.distances = \
  #  analyzer.construct_recon_adversarial_stimulus(adv_inputs, adv_targets, orig_recons.copy())
  #analyzer.all_adversarial_perturbations = []
  #for recon_mult_idx, adv_image_list in enumerate(analyzer.all_adversarial_images):
  #  analyzer.all_adversarial_perturbations.append([])
  #  for step_idx, adv_images in enumerate(adv_image_list):
  #    analyzer.all_adversarial_perturbations[recon_mult_idx].append(adv_images - adv_inputs)

In [ ]:
analyzer_idx = 0
batch_idx = 0
step_idx = -1
recon_idx = 0
fig, axes = plt.subplots(1, 7, figsize=(20,4))

ax = pf.clear_axis(axes[0])
ax.imshow(adv_inputs[batch_idx,...].reshape(28,28),
  cmap="Greys_r", vmin=0.0, vmax=1.0)
ax.set_title("Original\ninput image")
#ax = pf.clear_axis(axes[1])
#ax.imshow(analyzer_list[analyzer_idx].all_adversarial_images[0][0][batch_idx,...].reshape(28,28),
#  cmap="Greys_r", vmin=0.0, vmax=1.0)
#ax.set_title("Original\ninput image")

ax = pf.clear_axis(axes[2])
ax.imshow(orig_recons[batch_idx,...].reshape(28,28),
  cmap="Greys_r", vmin=0.0, vmax=1.0)
ax.set_title("Original\ninput recon image")
#ax = pf.clear_axis(axes[3])
#ax.imshow(analyzer_list[analyzer_idx].all_recons[0][0][batch_idx,...].reshape(28,28),
#  cmap="Greys_r", vmin=0.0, vmax=1.0)
#ax.set_title("Original\nrecon image")
#
#ax = pf.clear_axis(axes[4])
#ax.imshow(analyzer_list[analyzer_idx].all_adversarial_perturbations[0][step_idx][batch_idx,...].reshape(28,28),
#  cmap="Greys_r", vmin=0.0, vmax=1.0)
#ax.set_title("Adversarial\nperturbation image")
#
#ax = pf.clear_axis(axes[5])
#ax.imshow(analyzer_list[analyzer_idx].all_adversarial_images[0][step_idx][batch_idx,...].reshape(28,28),
#  cmap="Greys_r", vmin=0.0, vmax=1.0)
#ax.set_title("Adversarial\ninput image")
#
#ax = pf.clear_axis(axes[6])
#ax.imshow(analyzer_list[analyzer_idx].all_recons[0][step_idx][batch_idx,...].reshape(28,28),
#  cmap="Greys_r", vmin=0.0, vmax=1.0)
#ax.set_title("Adversarial\nrecon image")

plt.show()

In [ ]:
#These mses are in shape [num_recon_mults, num_iterations, num_batch]
fig, ax = plt.subplots(1, figsize=(4,4))

offset = 1
steps = analyzer_list[analyzer_idx].steps[offset:]

ax.plot(steps,
  np.array(analyzer_list[analyzer_idx].distances["input_adv_mses"])[recon_idx, offset:, batch_idx],
  label='input to perturbed', color='r')

#ax.plot(steps,
#  np.array(analyzer_list[analyzer_idx].distances["target_adv_mses"])[recon_idx, :, batch_idx][offset:],
#  label='target to perturbed', color='b')

#ax.plot(steps,
#  np.array(analyzer_list[analyzer_idx].distances["target_recon_mses"])[recon_idx, offset:, batch_idx],
#  label='target to recon', color='g')

#ax.plot(steps,
#  np.array(analyzer_list[analyzer_idx].distances["adv_recon_mses"])[recon_idx, :, batch_idx][offset:],
#  label='perturbed to recon', color='k')

ax.plot(steps,
  np.array(analyzer_list[analyzer_idx].distances["input_recon_mses"])[recon_idx, :, batch_idx][offset:],
  label='input to recon', color='c')

fig.legend(loc="upper right", bbox_to_anchor=(1.42, 0.86))
plt.show()

In [ ]:
num_bins = 100
similarities = []
for analyzer in analyzer_list:
  similarities.append(analyzer.adversarial_target_adv_cos_similarities[0][-1])
min_dist = np.minimum(0.0, np.min(similarities))
max_dist = np.maximum(1.0, np.max(similarities))
bins = np.linspace(min_dist, max_dist, num_bins)
fontsize = 18
line_alpha = 0.7

fig, ax = plt.subplots(1, figsize=(12,4))
for analyzer_idx, analyzer in enumerate(analyzer_list):
  cos_similarity = similarities[analyzer_idx]
  analyzer.dist_hist, bin_edges = np.histogram(cos_similarity, bins=bins)
  bin_left, bin_right = bin_edges[:-1], bin_edges[1:]
  bin_centers = bin_left + (bin_right - bin_left)/2
  
for analyzer_idx, analyzer in enumerate(analyzer_list):
  analyzer.dist_hist = analyzer.dist_hist / np.max([analyzer.dist_hist for analyzer in analyzer_list])
  ax.plot(bin_left, analyzer.dist_hist, alpha=line_alpha, linestyle="--", #color = 'k',
    drawstyle="steps-mid", label=analyzer.model_params.model_type.upper())
  
title = ("Cosine Similarity Between Target Image and Perturbation")
ax.set_title(title, fontsize=fontsize)
ax.set_xlabel("Cosine Similarity", fontsize=fontsize)
ax.set_ylabel("Normalized Count", fontsize=fontsize)
ax.tick_params("both", labelsize=fontsize)
ax.legend(fontsize=fontsize, fancybox=True, shadow=True, bbox_to_anchor=(0.8, 0.8))
plt.show()

for analyzer in analyzer_list:
  fig.savefig(analyzer.analysis_out_dir+"/vis/cosyne_similarity.png", transparent=True, bbox_inches="tight", pad_inches=0.01)

In [ ]:
diff_list = similarities[1] - similarities[0]

bins = np.linspace(np.min(diff_list), np.max(diff_list), num_bins)
dist_hist, bin_edges = np.histogram(diff_list, bins=bins)
dist_hist = dist_hist / np.max(dist_hist)
bin_left, bin_right = bin_edges[:-1], bin_edges[1:]
bin_centers = bin_left + (bin_right - bin_left)/2

fig, ax = plt.subplots(1, figsize=(12,4))
ax.bar(bin_centers, dist_hist, width=bin_right-bin_left, edgecolor='k')
ax.set_xlabel("", fontsize=fontsize)
ax.set_ylabel("Normalized Count", fontsize=fontsize)
  
title = ("Difference between Cosine Similarity Between Target Image and Perturbation")
ax.set_title(title, fontsize=fontsize)
ax.set_xlabel("Cosine Similarity Difference", fontsize=fontsize)
ax.set_ylabel("Normalized Count", fontsize=fontsize)
ax.tick_params("both", labelsize=fontsize)
plt.show()

In [ ]:
np.max(analyzer_list[analyzer_idx].all_adversarial_images[0][step_idx])

In [ ]:
np.all(analyzer_list[analyzer_idx].all_adversarial_images[0][step_idx][batch_idx,...] == 0.1

In [ ]:
np.min(analyzer_list[analyzer_idx].all_adversarial_images[0][step_idx][batch_idx,...])